In [32]:
import cat4py as cat
import numpy as np

In [33]:
shape = (10, 10)
a = np.arange(np.prod(shape), dtype='f4').reshape(shape)

In [36]:
pshape = (2, 5)
b = cat.empty(shape, pshape=pshape, dtype='f4')

In [37]:
for block, info in b.iter_write():
    block[:] = a[info.slice]

In [38]:
for block, info in b.iter_read():
    np.testing.assert_allclose(block, a[info.slice])